In [7]:
'''
Analysis on how the ownership of a token holder change
'''

'\nAnalysis on how the ownership of a token holder change\n'

In [8]:
from bs4 import BeautifulSoup
import urllib3

In [9]:
baseUrl='https://etherscan.io/'
funUrl_chart='token/tokenholderchart/'
tokenid='0x86fa049857e0209aa7d9e616f7eb3b3b78ecfdb0'

pool=urllib3.PoolManager()
r=pool.request('GET',baseUrl+funUrl_chart+tokenid)
webdata=r.data

soup = BeautifulSoup(webdata)
table = soup.find("table", attrs={"class":"table table-hover "})
headings = [th.get_text() for th in table.find("tr").find_all("th")]
datasets = []
for row in table.find_all("tr")[1:]:
    dataset = zip(headings, (td.get_text() for td in row.find_all("td")))
    datasets.append(dataset)
print datasets

/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[[(u'Rank', u'1'), (u'Address', u'0xd0a6e6c54dbc68db5db3a091b171a77407ff7ccf (EOSCrowdsale)'), (u'Quantity (Token)', u'663037530.298454685788795826'), (u'Percentage', u'66.3038%')], [(u'Rank', u'2'), (u'Address', u'0x00000000000000000000000000000000000000b1'), (u'Quantity (Token)', u'100000000'), (u'Percentage', u'10.0000%')], [(u'Rank', u'3'), (u'Address', u'0x42da8a05cb7ed9a43572b5ba1b8f82a0a6e263dc (yunbi_2)'), (u'Quantity (Token)', u'35160347.008970204988273462'), (u'Percentage', u'3.5160%')], [(u'Rank', u'4'), (u'Address', u'0xf4b51b14b9ee30dc37ec970b50a486f37686e2a8'), (u'Quantity (Token)', u'13926010.71600623992561125'), (u'Percentage', u'1.3926%')], [(u'Rank', u'5'), (u'Address', u'0x174443351e21d47ed9ab51517a301107d92ede64'), (u'Quantity (Token)', u'13732674.086746046713939295'), (u'Percentage', u'1.3733%')], [(u'Rank', u'6'), (u'Address', u'0xe2dd88165572932944ba72ab6ed143f9a9855a81'), (u'Quantity (Token)', u'4400000'), (u'Percentage', u'0.4400%')], [(u'Rank', u'7'), (u'Addre